In [ ]:
from bs4 import BeautifulSoup
import requests
import re
import json
from urllib.parse import urljoin

def scrape_mushroom_list(url):
    data = requests.get(url).text
    soup = BeautifulSoup(data, 'html.parser')

    # Trouver tous les liens vers les pages individuelles des champignons
    mushroom_links = soup.find_all('a', href=re.compile(r'/show\?n='))
    mushroom_urls = [urljoin('http://www.mushroom.world', link['href']) for link in mushroom_links]

    # Récupérer les informations pour chaque champignon
    mushrooms = [scrape_mushroom(link) for link in mushroom_urls]

    return mushrooms

def scrape_mushroom(url):
    data = requests.get(url).text
    soup = BeautifulSoup(data, 'html.parser')

    # Extraire les informations spécifiques pour un champignon individuel
    name_content = soup.find(class_="caption").find("b").contents
    names = re.sub('[^A-Za-z0-9( ]+', '', name_content[0]).split("(")
    names = [n.strip() for n in names]
    name1 = names[0]
    name2 = names[1] if len(names) > 1 else ''

    labels = soup.find_all(class_="labelus")
    labels = [label.contents[0] for label in labels]

    texts = soup.find_all(class_="textus")
    texts = [text.contents[0] for text in texts]

    description = soup.find(class_="longtextus").contents
    description = [re.sub('[^A-Za-z0-9,.<> ]+', '', str(d)).strip() for d in description]
    description = [re.sub('<b>', '', d) for d in description if (d != "") & (d != "<br>")]
    description.insert(0, 'General')
    description = dict(zip(description[0::2], description[1::2]))

    texts.append(description)
    assert len(labels) == len(texts)

    images = soup.find(id="mushroom-list").find_all(class_="image")
    image_urls = [urljoin('http://www.mushroom.world', image.a["href"]) for image in images]

    mushroom = dict(name1=name1, name2=name2, images=image_urls, info=dict())

    for i in range(len(labels)):
        mushroom["info"][labels[i]] = texts[i]

    return mushroom

if __name__ == '__main__':
    # Lien vers la liste de tous les champignons sur mushroom.world
    all_mushrooms_url = 'http://www.mushroom.world/mushrooms/namelist'

    # Récupérer les informations sur tous les champignons
    all_mushrooms = scrape_mushroom_list(all_mushrooms_url)

    # Afficher les informations (ou les enregistrer dans un fichier, etc.)
    print(json.dumps(all_mushrooms))